## Data 604 Final Project

Using SimPy, write a process simulation that includes waiting time (discrete event simulation). You may use any topic of interest to you. Write the simulation and all of the following in Jupyter.

**1. State the problem and its significance.**

This project simulates a restaurant that offers 2 services to customers:

* **Self Service Buffet**    
    _The customer is provided with a plate and serves themselves to the buffet_.
    
    
* **Table Service**  
   _The customer is seated and their food is served to them by a waiter_. 

2.  Provide a flow-chart model. 

3.  Simulate the process for the appropriate number of iterations (justify)

4.  Justify the validity of the model and discuss how you verified it.

5.  State  your conclusions/ findings from the model. 

6.  Generate appropriate graphs (more than one) to illustrate the results and provide a PowerPoint presentation to share with your colleagues.  Post this to the discussion. 